# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlite3
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download(['punkt', 'stopwords','wordnet', 'averaged_perceptron_tagger'])

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier


import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Birgit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Birgit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Birgit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Birgit\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponseTable', engine)

df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# define X features values
X = df['message']

# define Y - target values, being the values you want to try to predict
# skip the first four columns id, message, original and genre
Y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
    """
    Clean, normalize, tokenize and lemmonize the text
    
    Input:
        text: text that needs to bei transformed
        
    Output:
        token_final: cleaned, normalizend and lemmonized text tokens

    """
    
    # define url_regex as the regular expression to detect a url
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # Normalize text
    # Remove punctuation characters and convert to lowercase in one step
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # Tokenize text
    # Split text into words using NLTK
    token = word_tokenize(text)

    # Remove stop words
    token = [w for w in token if w not in stopwords.words("english")]

    # Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in token]

    # Lemmatize verbs by specifying pos
    token_final = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]

    return token_final



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# build pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Split data into random train and test subsets
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

# train classifier
pipeline_fit = pipeline.fit(X_train, Y_train)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# Predict test labels
Y_pred = pipeline_fit.predict(X_test)

In [8]:
# Print classification report on test data
print(classification_report(Y_test.values, Y_pred, target_names=Y.columns.values, zero_division=0))

                        precision    recall  f1-score   support

               related       0.81      0.96      0.88      3998
               request       0.88      0.42      0.57       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.79      0.61      0.69      2164
          medical_help       0.68      0.04      0.08       435
      medical_products       0.79      0.05      0.10       279
     search_and_rescue       0.75      0.02      0.04       136
              security       0.33      0.01      0.02        96
              military       0.73      0.05      0.09       158
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.27      0.42       335
                  food       0.89      0.42      0.58       584
               shelter       0.89      0.26      0.41       468
              clothing       0.90      0.13      0.23        70
                 money       0.88      

In [9]:
# show confusion matrix
labels = np.unique(Y_pred)
confusion_mat = confusion_matrix(Y_test.values.argmax(axis=1),Y_pred.argmax(axis=1),labels=labels)
accuracy = (Y_pred == Y_test).mean()

print("Labels:", labels)
print("Confusion Matrix:\n", confusion_mat)
print("Accuracy:\n", accuracy)

Labels: [0 1]
Confusion Matrix:
 [[5241    0]
 [   0    0]]
Accuracy:
 related                   0.801259
request                   0.892237
offer                     0.995422
aid_related               0.772649
medical_help              0.918940
medical_products          0.948884
search_and_rescue         0.974442
security                  0.981499
military                  0.970818
child_alone               1.000000
water                     0.951364
food                      0.930193
shelter                   0.931337
clothing                  0.988175
money                     0.979783
missing_people            0.988175
refugees                  0.967385
death                     0.957849
other_aid                 0.869540
infrastructure_related    0.935915
transport                 0.956704
buildings                 0.951936
electricity               0.978447
tools                     0.993324
hospitals                 0.990082
shops                     0.995232
aid_centers        

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# set parameters 
parameters = {'clf__estimator__n_estimators':[5]
                #'clf__estimator__n_estimators': [20, 50],
                #'clf__estimator__min_samples_split': [2, 4],
                #'clf__estimator__max_features': ['sqrt', 'log2']
                }

# new model with improved parameters
cv = GridSearchCV(pipeline, param_grid=parameters, cv = 2)


In [11]:
# Fit or train the classifier
cv.fit(X_train, Y_train)

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))]),
             param_grid={'clf__estimator__n_estimators': [5]})

In [12]:
# Get the prediction values from the grid search cross validator
Y_pred_cv = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
# Print classification report on test data
print(classification_report(Y_test.values, Y_pred_cv, target_names=Y.columns.values, zero_division=0))

                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      3998
               request       0.72      0.45      0.55       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.68      0.55      0.61      2164
          medical_help       0.54      0.14      0.22       435
      medical_products       0.59      0.16      0.25       279
     search_and_rescue       0.27      0.04      0.08       136
              security       0.00      0.00      0.00        96
              military       0.48      0.15      0.22       158
           child_alone       0.00      0.00      0.00         0
                 water       0.72      0.38      0.50       335
                  food       0.74      0.48      0.58       584
               shelter       0.69      0.31      0.42       468
              clothing       0.70      0.20      0.31        70
                 money       0.36      

In [14]:
# display a confusion matrix and accuracy score based on the model's predictions
labels_cv = np.unique(Y_pred_cv)
confusion_mat = confusion_matrix(Y_test.values.argmax(axis=1),Y_pred_cv.argmax(axis=1), labels=labels_cv)
accuracy = (Y_pred_cv == Y_test).mean()

print("Labels:", labels_cv)
print("Confusion Matrix:\n", confusion_mat)
print("Accuracy:\n", accuracy)
print("Best Parameters:", cv.best_params_)

Labels: [0 1]
Confusion Matrix:
 [[5087   27]
 [   0    0]]
Accuracy:
 related                   0.787717
request                   0.876597
offer                     0.995422
aid_related               0.708373
medical_help              0.918749
medical_products          0.949456
search_and_rescue         0.972153
security                  0.980545
military                  0.969483
child_alone               1.000000
water                     0.951173
food                      0.923326
shelter                   0.925997
clothing                  0.988175
money                     0.977494
missing_people            0.987412
refugees                  0.966431
death                     0.956704
other_aid                 0.861339
infrastructure_related    0.932863
transport                 0.953652
buildings                 0.949838
electricity               0.978257
tools                     0.993324
hospitals                 0.989319
shops                     0.995041
aid_centers        

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
# define a custom transformer class 
# it extracts features from text data by determining if the text starts with a verb or not.

class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            if len(pos_tags) > 1:
                
                first_word, first_tag = pos_tags[0]
            
                
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return 1
                else:
                    return 0
            else:
                return 0

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        X_tagged = pd.DataFrame(X_tagged)
        X_tagged = X_tagged.replace(np.nan, 0)
        
        return pd.DataFrame(X_tagged)

In [16]:
# define a feature union pipeline with improved parameters
pipeline_improved = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier(min_samples_split = 4)))
    ])


In [17]:
# Fit or train the classifier
pipeline_improved_fit = pipeline_improved.fit(X_train, Y_train)

In [18]:
# Predict test labels
Y_imp_pred = pipeline_improved_fit.predict(X_test)

In [19]:
# Print classification report on test data
print(classification_report(Y_test.values, Y_imp_pred, target_names=Y.columns.values, zero_division=0))

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      3998
               request       0.86      0.51      0.64       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.76      0.70      0.73      2164
          medical_help       0.71      0.10      0.18       435
      medical_products       0.82      0.08      0.15       279
     search_and_rescue       0.50      0.04      0.08       136
              security       0.67      0.02      0.04        96
              military       0.75      0.08      0.14       158
           child_alone       0.00      0.00      0.00         0
                 water       0.89      0.37      0.53       335
                  food       0.87      0.63      0.73       584
               shelter       0.84      0.40      0.54       468
              clothing       0.90      0.13      0.23        70
                 money       0.89      

In [20]:
# display a confusion matrix and accuracy score based on the model's predictions
labels_imp = np.unique(Y_imp_pred)
confusion_mat = confusion_matrix(Y_test.values.argmax(axis=1),Y_imp_pred.argmax(axis=1), labels=labels_imp)
accuracy = (Y_imp_pred == Y_test).mean()

print("Labels:", labels_imp)
print("Confusion Matrix:\n", confusion_mat)
print("Accuracy:\n", accuracy)
    

Labels: [0 1]
Confusion Matrix:
 [[5231    2]
 [   0    0]]
Accuracy:
 related                   0.825100
request                   0.901583
offer                     0.995422
aid_related               0.784284
medical_help              0.922182
medical_products          0.950219
search_and_rescue         0.974061
security                  0.981881
military                  0.971390
child_alone               1.000000
water                     0.957086
food                      0.948312
shelter                   0.939538
clothing                  0.988175
money                     0.979973
missing_people            0.987984
refugees                  0.968148
death                     0.958230
other_aid                 0.868396
infrastructure_related    0.935533
transport                 0.956895
buildings                 0.953080
electricity               0.978829
tools                     0.993324
hospitals                 0.990082
shops                     0.995232
aid_centers        

### 9. Export your model as a pickle file

In [21]:
#Save the model as a pickle file
pickle.dump(pipeline_improved_fit, open('classifier.pkl', "wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.